# Cuarto Examen Parcial (Práctico)
### Laboratorio de Aprendizaje Estadístico

José Armando Melchor Soto  | 745697

--- 




### **Librerías**

In [ ]:
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
import numpy as np
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
from sklearn import linear_model
import statsmodels.formula.api as smf
import warnings
from scipy import stats
from sklearn.model_selection import train_test_split, GridSearchCV,RandomizedSearchCV
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_curve
import xgboost as xgb 
from xgboost import  XGBRegressor, XGBClassifier, plot_importance as xgb_plot_importance, callback as xgb_callback
from catboost import CatBoostRegressor, CatBoostClassifier
import lightgbm as lgb
from lightgbm import LGBMRegressor,LGBMClassifier, plot_importance,early_stopping, log_evaluation
import shap 
import shap.plots
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from IPython.display import display




warnings.simplefilter(action='ignore', category=FutureWarning)

colors = ["cornflowerblue", "indianred", "darkseagreen", "plum", "dimgray"]

---

### **Dataset** 
Diabetes

In [ ]:
df = pd.read_csv('diabetes.csv')

### **Visualización del Dataset**

In [ ]:
df.head()

In [ ]:
df.info()

### **Explicación de las variables**

---

### **Dummies**

In [ ]:
df = pd.get_dummies(df, drop_first=True)

In [ ]:
df.head()

---

### **NaN's**

In [ ]:
pd.DataFrame(df.isnull().sum()).T

##### Si es que hay, usamos la mediana para rellenar

In [ ]:
'''
df['--'] = df['--'].fillna(df['--'].median())
'''

##### Comprobamos que ya no haya NaN's

In [ ]:
pd.DataFrame(df.isnull().sum()).T

---

### **Funciones**

##### Función Calibración 

In [ ]:
def calibracion(predicciones, y, bins):
    df_calibracion = pd.DataFrame({
        'predicciones': predicciones,
        'y': y
    })
    
    df_calibracion['bins'] = pd.qcut(df_calibracion['predicciones'], q=bins, labels=False) + 1
    gropuped = df_calibracion.groupby('bins').mean()
    
    plt.plot(gropuped.predicciones, gropuped.y, label="Modelo", marker="o")
    plt.plot(gropuped.predicciones, gropuped.predicciones, label="Real", marker="o")
    plt.grid()
    plt.legend()
    plt.xlabel("Prediccion")
    plt.ylabel("Real")

##### Función KMeans

In [ ]:
def kmeans(df, columnas, max_clusters=11):
    X = df[columnas].values

    inertias = []
    
    for k in range(1, max_clusters + 1):
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('kmeans', KMeans(n_clusters=k, random_state=42))
        ])
        pipeline.fit(X)
        inertia = pipeline.named_steps['kmeans'].inertia_
        inertias.append(inertia)

    plt.figure(figsize=(8, 6))
    plt.plot(range(1, max_clusters + 1), inertias, marker='o')
    plt.xlabel('Número de Clusters')
    plt.ylabel('Inercia')
    plt.title('Método del Codo')
    plt.grid(True)
    plt.show()


##### Función Clusters

In [ ]:
def clusters(df, clusters, comp_x='PC1', comp_y='PC2'):
    df = df.copy().dropna()

    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('kmeans', KMeans(n_clusters=clusters, random_state=42))
    ])

    pipeline.fit(df)
    df['cluster'] = pipeline.predict(df)

    pca = PCA(n_components=max(6, clusters))
    X_pca = pca.fit_transform(df)

    x_components = pd.DataFrame(X_pca, columns=[f'PC{i+1}' for i in range(X_pca.shape[1])])
    x_components['Cluster'] = df['cluster'] + 1

    plt.figure(figsize=(8, 6))
    plt.scatter(x_components[comp_x], x_components[comp_y], c=x_components['Cluster'])
    plt.xlabel(comp_x)
    plt.ylabel(comp_y)
    plt.title(f'Con {clusters} Clusters visualizados en {comp_x} y {comp_y}')
    plt.show()



##### Función PCA

In [ ]:
def pca(X, n_var):
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA())
    ])

    pipeline.fit(X)
    objeto_pca = pipeline.named_steps['pca']

    X_pca = pipeline.transform(X)
    x_components = pd.DataFrame(X_pca, columns=[f'PC{i+1}' for i in range(X_pca.shape[1])])

    plt.plot(objeto_pca.explained_variance_ratio_, marker='o')
    plt.grid(True)
    plt.show()

    return x_components.head()

##### Función PCA Varianza

In [ ]:
def var_pca(X, n_var):
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=n_var)) 
    ])

    pipeline.fit(X)
    objeto_pca = pipeline.named_steps['pca']

    X_pca = pipeline.transform(X)
    x_components = pd.DataFrame(X_pca, columns=[f'PC{i+1}' for i in range(X_pca.shape[1])])

    plt.plot(objeto_pca.explained_variance_ratio_, marker='o')
    plt.xlabel('Componente Principal')
    plt.ylabel('Varianza Explicada')
    plt.title('Varianza Explicada por Componente')
    plt.grid(True)
    plt.show()

    varianza = pd.DataFrame({
        f'PC{i+1}': objeto_pca.explained_variance_ratio_[i] for i in range(n_var)
    }, index=['Varianza']).T

    display(varianza)

    return x_components.head()




##### Función Gráfica n PCA

In [ ]:
def plot_pca(X, comp_x='PC1', comp_y='PC2'):
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA())
    ])

    pipeline.fit(X)
    objeto_pca = pipeline.named_steps['pca']

    X_pca = pipeline.transform(X)
    x_components = pd.DataFrame(X_pca, columns=[f'PC{i+1}' for i in range(X_pca.shape[1])])
    
    if comp_x not in x_components.columns or comp_y not in x_components.columns:
        raise ValueError(f"Componentes inválidas: {comp_x}, {comp_y}. Usa nombres como 'PC1', 'PC2', etc.")

    plt.figure(figsize=(8, 6))
    plt.scatter(x_components[comp_x], x_components[comp_y], alpha=1)
    plt.xlabel(comp_x)
    plt.ylabel(comp_y)
    plt.title(f'PCA - Proyección en {comp_x} y {comp_y}')
    plt.grid(True)
    plt.show()


##### Función XGBoost

In [ ]:
def xgboost(X_train, y_train, X_test, y_test):
    model_xgboost = XGBClassifier(
        n_estimators=1000,
        max_depth=5,
        learning_rate=0.1,
        reg_lambda=0.1,
        subsample=0.5,
        colsample_bytree=0.8,
        tree_method='hist',
        enable_categorical=True,
        use_label_encoder=False,
        eval_metric='auc'
    )
    model_xgboost.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=100)
    proba_xgboost = model_xgboost.predict_proba(X_test)[:, 1]

    auc_score = roc_auc_score(y_test, proba_xgboost)
   
    fpr, tpr, _ = roc_curve(y_test, proba_xgboost)
    plt.figure(figsize=(8,6))
    plt.plot(fpr, tpr, label=f'XGBoost (AUC = {auc_score:.3f})')
    plt.plot([0, 1], [0, 1], 'r--', label='Clasificador aleatorio')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Curva ROC - XGBoost')
    plt.legend()
    plt.grid(True)
    plt.show()

    juntar = pd.DataFrame({
        'real': y_test,
        'probabilidades': proba_xgboost
    })

    plt.figure(figsize=(10, 5))
    plt.hist(juntar[juntar.real == 1]['probabilidades'], density=True, alpha=0.5, label='1')
    plt.hist(juntar[juntar.real == 0]['probabilidades'], density=True, alpha=0.5, label='0')
    plt.title('Histograma de probabilidades XGBoost')
    plt.xlabel('Probabilidad')
    plt.ylabel('Densidad')
    plt.legend()
    plt.grid(True)
    plt.show()


    importancia = pd.DataFrame({
        'variables': X_train.columns,
        'importancia': model_xgboost.feature_importances_
    })
    importancia.sort_values('importancia', ascending=True, inplace=True)
    importancia0 = importancia.tail(5)
    importancia0.plot.barh(x='variables', y='importancia', figsize=(8,6))
    plt.xlabel('Importancia')
    plt.title('Top 5 Feature Importances')
    plt.grid(True)
    plt.show()
    
    print("AUC XGBoost:", auc_score)




##### Función XGBoost Shap values

In [ ]:
def shapXGB(X_train, y_train, X_test, y_test):

    model_xgboost = XGBClassifier(
        n_estimators=1000,
        max_depth=5,
        learning_rate=0.1,
        reg_lambda=0.1,
        subsample=0.5,
        colsample_bytree=0.8,
        tree_method='hist',
        enable_categorical=True,
        use_label_encoder=False,
        eval_metric='auc'
    )
    model_xgboost.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=100)

    proba_xgboost = model_xgboost.predict_proba(X_test)[:, 1]
    auc_score = roc_auc_score(y_test, proba_xgboost)
    print(f"AUC XGBoost: {auc_score:.4f}")

    sample_size = min(10000, len(X_test))
    X_shap = X_test.sample(sample_size, random_state=42)

    explainer = shap.TreeExplainer(model_xgboost)
    shap_values = explainer.shap_values(X_shap)

    shap.initjs()

    shap.waterfall_plot(shap.Explanation(
        values=shap_values[0],
        base_values=explainer.expected_value,
        data=X_shap.iloc[0],
        feature_names=X_shap.columns
    ))

    plt.title('Top 5 Variables más importantes (Barplot)')
    shap.summary_plot(shap_values, X_shap, plot_type="bar", max_display=5)

    plt.title('Top 10 Variables con mayor impacto (Dotplot)')
    shap.summary_plot(shap_values, X_shap, plot_type="dot", max_display=10)


---

### **Modelo**

Se pide que el modelo tenga: 

test set: 20%
random_state: 42

In [ ]:
target= 'diabetes'
X = df.drop(target, axis=1)
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

---

#### **Correlación del target con todas las variables**

In [ ]:
df.corr()['diabetes'].sort_values(ascending=False)

In [ ]:
df.drop('gender_Other', axis=1, inplace=True)

---

## **Modelos**

## ---

---

### **Conclusión**